# Importing Libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from z3 import *
from sklearn.cluster import DBSCAN
from scipy.spatial import ConvexHull, convex_hull_plot_2d
import sys
import warnings
warnings.filterwarnings("ignore")
import json

# FIXED PARAMETERS

In [2]:
NUM_ZONES = 5

# Clustering

In [3]:
# returns dbscan clusters
def dbscan(X, eps, min_samples):
    db = DBSCAN(eps=eps, min_samples=min_samples)
    cluster = db.fit(X)
    #y_pred = db.fit_predict(X)
    #plt.scatter(X[:,0], X[:,1],c=y_pred, cmap='Paired')
    #plt.title("DBSCAN")
    #print("noise", sum(cluster.labels_ ==-1), "regular", sum(cluster.labels_ !=-1))
    #print(np.unique(cluster.labels_))
    return cluster

# Convex Hull

In [4]:
# returns convex hull of points associated with zones
def convex_hull(zone, points):
        '''
        Parameters
        ----------
        points : Array
            A set of points.
            
        Returns
        -------
        vertices : Points
            Arranges convex hull vertices in counter-clockwise orientation.
        '''    
        
        hull = ConvexHull(points)
        
        
        #plt.plot(points[:,0], points[:,1], 'o')
        #plt.ylim(0,120)
        #for simplex in hull.simplices:
        #    plt.plot(points[simplex, 0], points[simplex, 1], 'k-')
            
        simplices = hull.simplices
        
        vertices = []
        for index in hull.vertices:
            vertices.append((points[index][0], zone, points[index][1]))
        vertices.append((points[hull.vertices[0]][0], zone, points[hull.vertices[0]][1]))
        
        return vertices

# Cluster Boundary Acquisition

In [5]:
def get_cluster(dataframe, eps, min_samples):
    list_cluster = []
    
    dataframe = dataframe[dataframe['Day'] <= 15] 
    
    for zone in range(NUM_ZONES):
        #plt.figure()
        mod_dataframe = pd.DataFrame()

        for i in range(len(dataframe)):
            if int(dataframe['Occupant\'s Zone'][i] == zone):
                mod_dataframe = mod_dataframe.append(dataframe.loc[i, ['Zone Arrival Time (Minute)', 'Stay Duration (Minute)']])

        features = np.empty([len(mod_dataframe), 2])
        features[:, 0:1] = mod_dataframe.loc[:, ['Zone Arrival Time (Minute)']].values
        features[:, 1:] = mod_dataframe.loc[:, ['Stay Duration (Minute)']].values
        cluster_model = dbscan(features, eps, min_samples)
        labels = cluster_model.labels_
        
        n_clusters = len(set(labels)) - (1 if -1 in labels else 0)
        for cluster in range(n_clusters):
            points = []
            for k in range(len(labels)):
                if labels[k] == cluster:
                    points.append(features[k])
            points = np.array(points)

            if len(points) >= 3:
                try:
                    vertices = convex_hull(zone, points)
                except:
                    print("Exception")
            else:
                print("zone", zone, "Problem Found!")

            list_cluster.append({"zone_id": zone, "cluster_id": cluster, "points": vertices})

    return list_cluster

# Clustering for House-A and Occupant-1

In [6]:
cleaned_duration_dataframe_house_A_occ_1 = pd.read_csv("../../data/cleaned/Cleaned-Dataframe_House-A_Occupant-1.csv")
cleaned_duration_dataframe_house_A_occ_1
clusters_house_A_occ_1 = get_cluster(cleaned_duration_dataframe_house_A_occ_1, 30, 3)
# saving clusters to file
with open('../../data/shatter/Clusters-House_A_Occ-1.txt', 'w') as f:
    f.write(json.dumps(clusters_house_A_occ_1))

# Clustering for House-A and Occupant-2

In [7]:
cleaned_duration_dataframe_house_A_occ_2 = pd.read_csv("../../data/cleaned/Cleaned-Dataframe_House-A_Occupant-2.csv")
cleaned_duration_dataframe_house_A_occ_2
clusters_house_A_occ_2 = get_cluster(cleaned_duration_dataframe_house_A_occ_2, 26, 3)
# saving clusters to file
with open('../../data/shatter/Clusters-House_A_Occ-2.txt', 'w') as f:
    f.write(json.dumps(clusters_house_A_occ_2))

# Clustering for House-B and Occupant-1

In [8]:
cleaned_duration_dataframe_house_B_occ_1 = pd.read_csv("../../data/cleaned/Cleaned-Dataframe_House-B_Occupant-1.csv")
cleaned_duration_dataframe_house_B_occ_1
clusters_house_B_occ_1 = get_cluster(cleaned_duration_dataframe_house_B_occ_1, 30, 3)
# saving clusters to file
with open('../../data/shatter/Clusters-House_B_Occ-1.txt', 'w') as f:
    f.write(json.dumps(clusters_house_B_occ_1))

# Clustering for House-B and Occupant-2

In [9]:
cleaned_duration_dataframe_house_B_occ_2 = pd.read_csv("../../data/cleaned/Cleaned-Dataframe_House-B_Occupant-2.csv")
cleaned_duration_dataframe_house_B_occ_2
clusters_house_B_occ_2 = get_cluster(cleaned_duration_dataframe_house_B_occ_2, 30, 3)
# saving clusters to file
with open('../../data/shatter/Clusters-House_B_Occ-2.txt', 'w') as f:
    f.write(json.dumps(clusters_house_B_occ_2))

Exception
Exception


# Range Claculation Function

In [10]:
def is_left( x, y, init_x, init_y, final_x, final_y):
     return ((final_x - init_x)*(y - init_y) - (final_y - init_y)*(x - init_x)) >= 0

def range_calculation(list_cluster):
    num_timeslots = 1440
    list_time_min = [[[] for j in range(num_timeslots)] for i in range(NUM_ZONES + 1)]
    list_time_max = [[[] for j in range(num_timeslots)] for i in range(NUM_ZONES + 1)]

    for i in range(len(list_cluster)):
        zone_id = list_cluster[i]["zone_id"]
        min_x_range = 1440
        max_x_range = 0

        ##################################################################
        ##################### Zone Constraints ###########################
        ##################################################################
        x = Int('x')
        y = Int('y')

        points = list_cluster[i]["points"]
        zone_constraints = []

        and_constraints = []
        for j in range(len(points) - 1):
            and_constraints.append(is_left(x, y, points[j][0], points[j][2], points[j + 1][0], points[j + 1][2]))

        zone_constraints.append(And(and_constraints))

        #print(zone_constraints)
        ####### Minimum value of X range #######
        o = Optimize()
        o.add(zone_constraints)
        o.minimize(x)
        o.check()
        
        min_x_range = int(str(o.model()[x]))

        ####### Maximum value of X range #######
        o = Optimize()
        o.add(zone_constraints)
        o.maximize(x)
        o.check()
        #print(o.model()[x])

        max_x_range = int(str(o.model()[x]))
        
        
        for j in range(min_x_range, max_x_range):
            ####### Minimum value of Y range #######
            o = Optimize()
            o.add(zone_constraints)
            o.add(x == j)
            o.minimize(y)
            o.check()

            min_y_range = o.model()[y]
            if min_y_range == None:
                min_y_range = 0

            ####### Maximum value of Y range #######
            o = Optimize()
            o.add(zone_constraints)
            o.add(x == j)
            o.maximize(y)
            o.check()

            max_y_range = o.model()[y]
            if max_y_range == None:
                max_y_range = 0
                
            list_time_min[zone_id][j].append(int(str(min_y_range)))
            list_time_max[zone_id][j].append(int(str(max_y_range)))
    return list_time_min, list_time_max

# Attack Scheduling Function

In [11]:
def attack_scheduling(list_cluster, num_timeslots, start_time, final_time):
    list_time_min, list_time_max = range_calculation(list_cluster)
    num_timeslots = interval = 10

    prev_stay = 1
    prev_schedule = -1
    ultimate_cost = 0
    final_schedule = schedule = np.zeros((final_time - start_time))

    for init_time in range(start_time, final_time, interval):
        cost = [Int( 'cost_' + str(i)) for i in range(NUM_ZONES)]
        zones = [Int( 'zones_' + str(i)) for i in range(NUM_ZONES)]

        schedule = [Int( 'schedule_' + str(i)) for i in range(init_time, init_time + interval)]
        stay = [Int( 'stay_' + str(i)) for i in range(num_timeslots)]
        slot_cost = [Int( 'slot_cost_' + str(i)) for i in range(num_timeslots)]

        total_cost = Int('total_cost')

        o = Optimize()
        o.add(cost[0] == 0)
        o.add(cost[1] == 1)
        o.add(cost[2] == 2)
        o.add(cost[3] == 4)
        o.add(cost[4] == 3)


        ############################################################################
        ################## schedule should be withing a valid zone #################
        ############################################################################
        for t in range(len(schedule)):
            or_constraints = []
            for z in range(NUM_ZONES):
                or_constraints.append(schedule[t] == z)
            o.add(Or(or_constraints))

        ###############################################################################################################
        ################## if zone stay threshdold in current time is 0, do not schedule to that zone #################
        ###############################################################################################################

        # base case
        for z in range(1, NUM_ZONES):
            if list_time_min[z][init_time] == [] or (len(list_time_max[z][init_time]) == 1 and list_time_max[z][init_time][0] == 0):
                o.add(Implies(schedule[0] != prev_schedule, schedule[t] != z))

        for t in range(1, len(schedule)):
            for z in range(1, NUM_ZONES):
                if list_time_min[z][init_time + t] == [] or (len(list_time_max[z][init_time + t]) == 1 and list_time_max[z][init_time + t][0] == 0):
                    o.add(Implies(schedule[t] != schedule[t - 1], schedule[t] != z))

        #######################################################################################################################
        ############################################ constraints of stay ######################################################
        #######################################################################################################################
        ######## base case for time 0 ############
        if init_time == 0:
            o.add(stay[0] == 1)
        else:
            o.add(Implies(schedule[0] == prev_schedule, stay[0] == prev_stay + 1))
            o.add(Implies(Not(schedule[0] == prev_schedule), stay[0] == 1))

        for t in range(1, len(schedule)):
            continue_staying = (schedule[t] == schedule[t - 1])
            increment_stay = (stay[t] == stay[t - 1] + 1)
            reset_stay = (stay[t] == 1)

            o.add(Implies(continue_staying, increment_stay))
            o.add(Implies(Not(continue_staying), reset_stay))

        #######################################################################################################################
        ############ move to a zone different that previous timeslot if stay > max threshold in previous timeslot #############
        #######################################################################################################################
        ######## base case for time 0 ############
        o.add(Implies(prev_stay == max(list_time_max[prev_schedule][init_time - prev_stay], default=0), schedule[0] != prev_schedule))

        for t in range(1, len(schedule)):
            max_stay_threshold = 0
            for z in range(1, NUM_ZONES):
                for p_t in range(1, init_time + len(schedule)):
                    continue_staying = (schedule[t] == schedule[t - 1])
                    o.add(Implies(And(schedule[t - 1] == z, stay[t - 1] == p_t, p_t == max(list_time_max[z][init_time + t - p_t], default=0)), Not(continue_staying)))

        #######################################################################################################################
        # must stay in the zone same as the previous timeslot if stay < max && stay is in previous timeslot is out of cluster #
        #######################################################################################################################
        ######## base case for time 0 ############
        ranges_stay_constraints = []
        for k in range(len(list_time_min[prev_schedule][init_time - prev_stay])):
            ranges_stay_constraints.append(And(prev_stay >= list_time_min[prev_schedule][init_time - prev_stay][k], prev_stay <= list_time_max[prev_schedule][init_time - prev_stay][k]))            

        if init_time != 0:
            o.add(Implies(Not(Or(ranges_stay_constraints)), schedule[0] == prev_schedule))

        for t in range(1, len(schedule)):
            for z in range(1, NUM_ZONES):
                for p_t in range(1, init_time + t + 1):
                    ranges_stay_constraints = []
                    for k in range(len(list_time_min[z][init_time + t - p_t])):
                        ranges_stay_constraints.append(And(p_t >= list_time_min[z][init_time + t - p_t][k], p_t <= list_time_max[z][init_time + t - p_t][k]))            

                    continue_staying = (schedule[t] == schedule[t - 1])
                    o.add(Implies(And(schedule[t - 1] == z, stay[t - 1] == p_t, Not(Or(ranges_stay_constraints))), continue_staying))

        for t in range(len(schedule)):
            for z in range(NUM_ZONES):
                o.add(Implies(schedule[t] == z, slot_cost[t] == cost[z]))

        o.add(total_cost == Sum(slot_cost))

        #o.add(schedule[0] == 3)

        o.maximize(total_cost)

        print(o.check())
        print(init_time, o.model()[total_cost])
        ultimate_cost += int(str(o.model()[total_cost]))

        print("Ulimate Cost", init_time, ultimate_cost)

        print("Num_timeslot", num_timeslots, "Optimal_cost", o.model()[total_cost])

        for t in range(num_timeslots):
            print("schedule_" + str(init_time + t), o.model()[schedule[t]])
            final_schedule[init_time + t] = int(str(o.model()[schedule[t]]))

        for t in range(num_timeslots):
            print("stay_" + str(init_time + t), o.model()[stay[t]])

        prev_schedule = int(str(o.model()[schedule[-1]]))
        prev_stay = int(str(o.model()[stay[-1]]))
    return final_schedule

# Greedy Scheduling Function

In [12]:
def greedy_schedule(list_cluster, num_timeslots):
    current_timeslot = 0
    final_timeslot = 0

    alternate_schedule = np.zeros((num_timeslots))
    list_time_min, list_time_max = range_calculation(list_cluster)
    while final_timeslot <= 1439:
        for zone in [3, 4, 2, 1, 0]:
            max_list = list_time_max[zone][current_timeslot]
            for m in range(len(max_list)):
                max_list[m] = int(str(max_list[m]))
            if max_list == []:
                if zone == 0:
                    print("Critical", current_timeslot)
                    final_timeslot = current_timeslot + 1
                    alternate_schedule[current_timeslot: final_timeslot] = 0
                    current_timeslot = final_timeslot
                continue
            max_val = max(max_list)
            if max_val == 0:
                final_timeslot = current_timeslot + 1
                alternate_schedule[current_timeslot: final_timeslot] = 0
                current_timeslot = final_timeslot
            if max_val != 0:
                final_timeslot = current_timeslot + max_val
                print("Here", current_timeslot, final_timeslot)
                alternate_schedule[current_timeslot: final_timeslot] = zone
                print("selected zone", zone)
                current_timeslot = final_timeslot
                break
    return alternate_schedule

# Variable Parameters

In [13]:
num_timeslots = interval = 10
start_time = 0
final_time = 1440

# House-A Occupant-1 Attack Schedule

In [14]:
schedule_house_A_occ_1 = attack_scheduling(clusters_house_A_occ_1, num_timeslots, start_time, final_time)
pd.DataFrame(schedule_house_A_occ_1, columns = ["Schedule"], dtype='int64').to_csv("../../data/shatter/Partial_DBSCAN_House-A_Occ-1_SHATTER_Schedule.csv", index = False)

sat
0 40
Ulimate Cost 0 40
Num_timeslot 10 Optimal_cost 40
schedule_0 3
schedule_1 3
schedule_2 3
schedule_3 3
schedule_4 3
schedule_5 3
schedule_6 3
schedule_7 3
schedule_8 3
schedule_9 3
stay_0 1
stay_1 2
stay_2 3
stay_3 4
stay_4 5
stay_5 6
stay_6 7
stay_7 8
stay_8 9
stay_9 10
sat
10 36
Ulimate Cost 10 76
Num_timeslot 10 Optimal_cost 36
schedule_10 3
schedule_11 3
schedule_12 3
schedule_13 3
schedule_14 3
schedule_15 3
schedule_16 0
schedule_17 3
schedule_18 3
schedule_19 3
stay_10 11
stay_11 12
stay_12 13
stay_13 14
stay_14 15
stay_15 16
stay_16 1
stay_17 1
stay_18 2
stay_19 3
sat
20 24
Ulimate Cost 20 100
Num_timeslot 10 Optimal_cost 24
schedule_20 3
schedule_21 3
schedule_22 2
schedule_23 2
schedule_24 2
schedule_25 2
schedule_26 2
schedule_27 2
schedule_28 2
schedule_29 2
stay_20 4
stay_21 5
stay_22 1
stay_23 2
stay_24 3
stay_25 4
stay_26 5
stay_27 6
stay_28 7
stay_29 8
sat
30 24
Ulimate Cost 30 124
Num_timeslot 10 Optimal_cost 24
schedule_30 2
schedule_31 2
schedule_32 2
schedul

sat
260 10
Ulimate Cost 260 549
Num_timeslot 10 Optimal_cost 10
schedule_260 1
schedule_261 1
schedule_262 1
schedule_263 1
schedule_264 1
schedule_265 1
schedule_266 1
schedule_267 1
schedule_268 1
schedule_269 1
stay_260 103
stay_261 104
stay_262 105
stay_263 106
stay_264 107
stay_265 108
stay_266 109
stay_267 110
stay_268 111
stay_269 112
sat
270 10
Ulimate Cost 270 559
Num_timeslot 10 Optimal_cost 10
schedule_270 1
schedule_271 1
schedule_272 1
schedule_273 1
schedule_274 1
schedule_275 1
schedule_276 1
schedule_277 1
schedule_278 1
schedule_279 1
stay_270 113
stay_271 114
stay_272 115
stay_273 116
stay_274 117
stay_275 118
stay_276 119
stay_277 120
stay_278 121
stay_279 122
sat
280 10
Ulimate Cost 280 569
Num_timeslot 10 Optimal_cost 10
schedule_280 1
schedule_281 1
schedule_282 1
schedule_283 1
schedule_284 1
schedule_285 1
schedule_286 1
schedule_287 1
schedule_288 1
schedule_289 1
stay_280 123
stay_281 124
stay_282 125
stay_283 126
stay_284 127
stay_285 128
stay_286 129
stay_28

sat
500 10
Ulimate Cost 500 789
Num_timeslot 10 Optimal_cost 10
schedule_500 1
schedule_501 1
schedule_502 1
schedule_503 1
schedule_504 1
schedule_505 1
schedule_506 1
schedule_507 1
schedule_508 1
schedule_509 1
stay_500 343
stay_501 344
stay_502 345
stay_503 346
stay_504 347
stay_505 348
stay_506 349
stay_507 350
stay_508 351
stay_509 352
sat
510 10
Ulimate Cost 510 799
Num_timeslot 10 Optimal_cost 10
schedule_510 1
schedule_511 1
schedule_512 1
schedule_513 1
schedule_514 1
schedule_515 1
schedule_516 1
schedule_517 1
schedule_518 1
schedule_519 1
stay_510 353
stay_511 354
stay_512 355
stay_513 356
stay_514 357
stay_515 358
stay_516 359
stay_517 360
stay_518 361
stay_519 362
sat
520 10
Ulimate Cost 520 809
Num_timeslot 10 Optimal_cost 10
schedule_520 1
schedule_521 1
schedule_522 1
schedule_523 1
schedule_524 1
schedule_525 1
schedule_526 1
schedule_527 1
schedule_528 1
schedule_529 1
stay_520 363
stay_521 364
stay_522 365
stay_523 366
stay_524 367
stay_525 368
stay_526 369
stay_52

sat
750 30
Ulimate Cost 750 1523
Num_timeslot 10 Optimal_cost 30
schedule_750 4
schedule_751 4
schedule_752 4
schedule_753 4
schedule_754 4
schedule_755 4
schedule_756 4
schedule_757 4
schedule_758 4
schedule_759 4
stay_750 1
stay_751 2
stay_752 3
stay_753 4
stay_754 5
stay_755 6
stay_756 7
stay_757 8
stay_758 9
stay_759 10
sat
760 23
Ulimate Cost 760 1546
Num_timeslot 10 Optimal_cost 23
schedule_760 0
schedule_761 4
schedule_762 4
schedule_763 4
schedule_764 4
schedule_765 4
schedule_766 2
schedule_767 2
schedule_768 2
schedule_769 2
stay_760 1
stay_761 1
stay_762 2
stay_763 3
stay_764 4
stay_765 5
stay_766 1
stay_767 2
stay_768 3
stay_769 4
sat
770 20
Ulimate Cost 770 1566
Num_timeslot 10 Optimal_cost 20
schedule_770 2
schedule_771 2
schedule_772 2
schedule_773 2
schedule_774 2
schedule_775 2
schedule_776 2
schedule_777 2
schedule_778 2
schedule_779 2
stay_770 5
stay_771 6
stay_772 7
stay_773 8
stay_774 9
stay_775 10
stay_776 11
stay_777 12
stay_778 13
stay_779 14
sat
780 28
Ulimate 

sat
1000 40
Ulimate Cost 1000 2338
Num_timeslot 10 Optimal_cost 40
schedule_1000 3
schedule_1001 3
schedule_1002 3
schedule_1003 3
schedule_1004 3
schedule_1005 3
schedule_1006 3
schedule_1007 3
schedule_1008 3
schedule_1009 3
stay_1000 26
stay_1001 27
stay_1002 28
stay_1003 29
stay_1004 30
stay_1005 31
stay_1006 32
stay_1007 33
stay_1008 34
stay_1009 35
sat
1010 40
Ulimate Cost 1010 2378
Num_timeslot 10 Optimal_cost 40
schedule_1010 3
schedule_1011 3
schedule_1012 3
schedule_1013 3
schedule_1014 3
schedule_1015 3
schedule_1016 3
schedule_1017 3
schedule_1018 3
schedule_1019 3
stay_1010 36
stay_1011 37
stay_1012 38
stay_1013 39
stay_1014 40
stay_1015 41
stay_1016 42
stay_1017 43
stay_1018 44
stay_1019 45
sat
1020 36
Ulimate Cost 1020 2414
Num_timeslot 10 Optimal_cost 36
schedule_1020 0
schedule_1021 3
schedule_1022 3
schedule_1023 3
schedule_1024 3
schedule_1025 3
schedule_1026 3
schedule_1027 3
schedule_1028 3
schedule_1029 3
stay_1020 1
stay_1021 1
stay_1022 2
stay_1023 3
stay_1024 4

sat
1240 40
Ulimate Cost 1240 3121
Num_timeslot 10 Optimal_cost 40
schedule_1240 3
schedule_1241 3
schedule_1242 3
schedule_1243 3
schedule_1244 3
schedule_1245 3
schedule_1246 3
schedule_1247 3
schedule_1248 3
schedule_1249 3
stay_1240 40
stay_1241 41
stay_1242 42
stay_1243 43
stay_1244 44
stay_1245 45
stay_1246 46
stay_1247 47
stay_1248 48
stay_1249 49
sat
1250 40
Ulimate Cost 1250 3161
Num_timeslot 10 Optimal_cost 40
schedule_1250 3
schedule_1251 3
schedule_1252 3
schedule_1253 3
schedule_1254 3
schedule_1255 3
schedule_1256 3
schedule_1257 3
schedule_1258 3
schedule_1259 3
stay_1250 50
stay_1251 51
stay_1252 52
stay_1253 53
stay_1254 54
stay_1255 55
stay_1256 56
stay_1257 57
stay_1258 58
stay_1259 59
sat
1260 40
Ulimate Cost 1260 3201
Num_timeslot 10 Optimal_cost 40
schedule_1260 3
schedule_1261 3
schedule_1262 3
schedule_1263 3
schedule_1264 3
schedule_1265 3
schedule_1266 3
schedule_1267 3
schedule_1268 3
schedule_1269 3
stay_1260 60
stay_1261 61
stay_1262 62
stay_1263 63
stay_12

# House-A Occupant-1 Greedy Schedule

In [15]:
greedy_schedule_house_A_occ_1 = greedy_schedule(clusters_house_A_occ_1, 1440)
pd.DataFrame(greedy_schedule_house_A_occ_1, columns = ["Schedule"], dtype='int64').to_csv("../../data/shatter/Partial_DBSCAN_House-A_Occ-1_Greedy_Schedule.csv", index = False)

Here 0 18
selected zone 3
Here 18 22
selected zone 3
Here 22 98
selected zone 2
Here 98 106
selected zone 4
Here 106 114
selected zone 4
Here 114 122
selected zone 4
Here 122 131
selected zone 4
Here 131 139
selected zone 4
Here 139 146
selected zone 4
Here 146 153
selected zone 4
Here 153 157
selected zone 4
Here 157 165
selected zone 0
Here 165 580
selected zone 1
Here 580 620
selected zone 3
Here 620 624
selected zone 3
Here 624 686
selected zone 3
Here 686 702
selected zone 4
Here 702 719
selected zone 3
Here 719 738
selected zone 4
Here 738 757
selected zone 4
Here 757 765
selected zone 4
Here 765 793
selected zone 2
Here 793 819
selected zone 3
Here 819 888
selected zone 2
Here 888 915
selected zone 4
Here 915 950
selected zone 3
Here 950 995
selected zone 3
Here 995 1043
selected zone 3
Here 1043 1044
selected zone 4
Here 1044 1045
selected zone 4
Here 1045 1046
selected zone 4
Here 1046 1047
selected zone 4
Here 1047 1048
selected zone 4
Here 1048 1049
selected zone 4
Here 1049

# House-A Occupant-2 Attack Schedule

In [16]:
schedule_house_A_occ_2 = attack_scheduling(clusters_house_A_occ_2, num_timeslots, start_time, final_time)
pd.DataFrame(schedule_house_A_occ_2, columns = ["Schedule"], dtype='int64').to_csv("../../data/shatter/Partial_DBSCAN_House-A_Occ-2_SHATTER_Schedule.csv", index = False)

sat
0 20
Ulimate Cost 0 20
Num_timeslot 10 Optimal_cost 20
schedule_0 2
schedule_1 2
schedule_2 2
schedule_3 2
schedule_4 2
schedule_5 2
schedule_6 2
schedule_7 2
schedule_8 2
schedule_9 2
stay_0 1
stay_1 2
stay_2 3
stay_3 4
stay_4 5
stay_5 6
stay_6 7
stay_7 8
stay_8 9
stay_9 10
sat
10 20
Ulimate Cost 10 40
Num_timeslot 10 Optimal_cost 20
schedule_10 2
schedule_11 2
schedule_12 2
schedule_13 2
schedule_14 2
schedule_15 2
schedule_16 2
schedule_17 2
schedule_18 2
schedule_19 2
stay_10 11
stay_11 12
stay_12 13
stay_13 14
stay_14 15
stay_15 16
stay_16 17
stay_17 18
stay_18 19
stay_19 20
sat
20 20
Ulimate Cost 20 60
Num_timeslot 10 Optimal_cost 20
schedule_20 2
schedule_21 2
schedule_22 2
schedule_23 2
schedule_24 2
schedule_25 2
schedule_26 2
schedule_27 2
schedule_28 2
schedule_29 2
stay_20 21
stay_21 22
stay_22 23
stay_23 24
stay_24 25
stay_25 26
stay_26 27
stay_27 28
stay_28 29
stay_29 30
sat
30 24
Ulimate Cost 30 84
Num_timeslot 10 Optimal_cost 24
schedule_30 2
schedule_31 2
schedule_

sat
260 0
Ulimate Cost 260 502
Num_timeslot 10 Optimal_cost 0
schedule_260 0
schedule_261 0
schedule_262 0
schedule_263 0
schedule_264 0
schedule_265 0
schedule_266 0
schedule_267 0
schedule_268 0
schedule_269 0
stay_260 89
stay_261 90
stay_262 91
stay_263 92
stay_264 93
stay_265 94
stay_266 95
stay_267 96
stay_268 97
stay_269 98
sat
270 0
Ulimate Cost 270 502
Num_timeslot 10 Optimal_cost 0
schedule_270 0
schedule_271 0
schedule_272 0
schedule_273 0
schedule_274 0
schedule_275 0
schedule_276 0
schedule_277 0
schedule_278 0
schedule_279 0
stay_270 99
stay_271 100
stay_272 101
stay_273 102
stay_274 103
stay_275 104
stay_276 105
stay_277 106
stay_278 107
stay_279 108
sat
280 0
Ulimate Cost 280 502
Num_timeslot 10 Optimal_cost 0
schedule_280 0
schedule_281 0
schedule_282 0
schedule_283 0
schedule_284 0
schedule_285 0
schedule_286 0
schedule_287 0
schedule_288 0
schedule_289 0
stay_280 109
stay_281 110
stay_282 111
stay_283 112
stay_284 113
stay_285 114
stay_286 115
stay_287 116
stay_288 11

sat
510 30
Ulimate Cost 510 715
Num_timeslot 10 Optimal_cost 30
schedule_510 4
schedule_511 4
schedule_512 4
schedule_513 4
schedule_514 4
schedule_515 4
schedule_516 4
schedule_517 4
schedule_518 4
schedule_519 4
stay_510 3
stay_511 4
stay_512 5
stay_513 6
stay_514 7
stay_515 8
stay_516 9
stay_517 10
stay_518 11
stay_519 12
sat
520 30
Ulimate Cost 520 745
Num_timeslot 10 Optimal_cost 30
schedule_520 4
schedule_521 4
schedule_522 4
schedule_523 4
schedule_524 4
schedule_525 4
schedule_526 4
schedule_527 4
schedule_528 4
schedule_529 4
stay_520 13
stay_521 14
stay_522 15
stay_523 16
stay_524 17
stay_525 18
stay_526 19
stay_527 20
stay_528 21
stay_529 22
sat
530 10
Ulimate Cost 530 755
Num_timeslot 10 Optimal_cost 10
schedule_530 1
schedule_531 1
schedule_532 1
schedule_533 1
schedule_534 1
schedule_535 1
schedule_536 1
schedule_537 1
schedule_538 1
schedule_539 1
stay_530 1
stay_531 2
stay_532 3
stay_533 4
stay_534 5
stay_535 6
stay_536 7
stay_537 8
stay_538 9
stay_539 10
sat
540 30
Uli

sat
760 0
Ulimate Cost 760 980
Num_timeslot 10 Optimal_cost 0
schedule_760 0
schedule_761 0
schedule_762 0
schedule_763 0
schedule_764 0
schedule_765 0
schedule_766 0
schedule_767 0
schedule_768 0
schedule_769 0
stay_760 144
stay_761 145
stay_762 146
stay_763 147
stay_764 148
stay_765 149
stay_766 150
stay_767 151
stay_768 152
stay_769 153
sat
770 0
Ulimate Cost 770 980
Num_timeslot 10 Optimal_cost 0
schedule_770 0
schedule_771 0
schedule_772 0
schedule_773 0
schedule_774 0
schedule_775 0
schedule_776 0
schedule_777 0
schedule_778 0
schedule_779 0
stay_770 154
stay_771 155
stay_772 156
stay_773 157
stay_774 158
stay_775 159
stay_776 160
stay_777 161
stay_778 162
stay_779 163
sat
780 0
Ulimate Cost 780 980
Num_timeslot 10 Optimal_cost 0
schedule_780 0
schedule_781 0
schedule_782 0
schedule_783 0
schedule_784 0
schedule_785 0
schedule_786 0
schedule_787 0
schedule_788 0
schedule_789 0
stay_780 164
stay_781 165
stay_782 166
stay_783 167
stay_784 168
stay_785 169
stay_786 170
stay_787 171


sat
1010 20
Ulimate Cost 1010 1084
Num_timeslot 10 Optimal_cost 20
schedule_1010 2
schedule_1011 2
schedule_1012 2
schedule_1013 2
schedule_1014 2
schedule_1015 2
schedule_1016 2
schedule_1017 2
schedule_1018 2
schedule_1019 2
stay_1010 3
stay_1011 4
stay_1012 5
stay_1013 6
stay_1014 7
stay_1015 8
stay_1016 9
stay_1017 10
stay_1018 11
stay_1019 12
sat
1020 20
Ulimate Cost 1020 1104
Num_timeslot 10 Optimal_cost 20
schedule_1020 2
schedule_1021 2
schedule_1022 2
schedule_1023 2
schedule_1024 2
schedule_1025 2
schedule_1026 2
schedule_1027 2
schedule_1028 2
schedule_1029 2
stay_1020 13
stay_1021 14
stay_1022 15
stay_1023 16
stay_1024 17
stay_1025 18
stay_1026 19
stay_1027 20
stay_1028 21
stay_1029 22
sat
1030 18
Ulimate Cost 1030 1122
Num_timeslot 10 Optimal_cost 18
schedule_1030 2
schedule_1031 2
schedule_1032 0
schedule_1033 2
schedule_1034 2
schedule_1035 2
schedule_1036 2
schedule_1037 2
schedule_1038 2
schedule_1039 2
stay_1030 23
stay_1031 24
stay_1032 1
stay_1033 1
stay_1034 2
stay

sat
1250 40
Ulimate Cost 1250 1644
Num_timeslot 10 Optimal_cost 40
schedule_1250 3
schedule_1251 3
schedule_1252 3
schedule_1253 3
schedule_1254 3
schedule_1255 3
schedule_1256 3
schedule_1257 3
schedule_1258 3
schedule_1259 3
stay_1250 58
stay_1251 59
stay_1252 60
stay_1253 61
stay_1254 62
stay_1255 63
stay_1256 64
stay_1257 65
stay_1258 66
stay_1259 67
sat
1260 22
Ulimate Cost 1260 1666
Num_timeslot 10 Optimal_cost 22
schedule_1260 3
schedule_1261 2
schedule_1262 2
schedule_1263 2
schedule_1264 2
schedule_1265 2
schedule_1266 2
schedule_1267 2
schedule_1268 2
schedule_1269 2
stay_1260 68
stay_1261 1
stay_1262 2
stay_1263 3
stay_1264 4
stay_1265 5
stay_1266 6
stay_1267 7
stay_1268 8
stay_1269 9
sat
1270 20
Ulimate Cost 1270 1686
Num_timeslot 10 Optimal_cost 20
schedule_1270 2
schedule_1271 2
schedule_1272 2
schedule_1273 2
schedule_1274 2
schedule_1275 2
schedule_1276 2
schedule_1277 2
schedule_1278 2
schedule_1279 2
stay_1270 10
stay_1271 11
stay_1272 12
stay_1273 13
stay_1274 14
sta

# House-A Occupant-2 Greedy Schedule

In [17]:
greedy_schedule_house_A_occ_2 = greedy_schedule(clusters_house_A_occ_2, 1440)
pd.DataFrame(greedy_schedule_house_A_occ_2, columns = ["Schedule"], dtype='int64').to_csv("../../data/shatter/Partial_DBSCAN_House-A_Occ-2_Greedy_Schedule.csv", index = False)

Here 0 41
selected zone 2
Here 41 55
selected zone 4
Here 55 76
selected zone 1
Here 76 94
selected zone 1
Here 94 125
selected zone 3
Here 125 154
selected zone 3
Here 154 171
selected zone 4
Critical 171
Critical 172
Critical 173
Critical 174
Critical 175
Critical 176
Critical 177
Critical 178
Critical 179
Critical 180
Critical 181
Critical 182
Critical 183
Critical 184
Critical 185
Critical 186
Critical 187
Critical 188
Critical 189
Critical 190
Critical 191
Critical 192
Critical 193
Critical 194
Critical 195
Critical 196
Critical 197
Critical 198
Critical 199
Critical 200
Critical 201
Critical 202
Critical 203
Critical 204
Critical 205
Critical 206
Critical 207
Critical 208
Critical 209
Critical 210
Critical 211
Critical 212
Critical 213
Critical 214
Critical 215
Critical 216
Critical 217
Critical 218
Critical 219
Critical 220
Critical 221
Critical 222
Critical 223
Critical 224
Critical 225
Critical 226
Critical 227
Critical 228
Critical 229
Critical 230
Critical 231
Critical 232
C

# House-B Occupant-1 Attack Schedule

In [18]:
schedule_house_B_occ_1 = attack_scheduling(clusters_house_B_occ_1, num_timeslots, start_time, final_time)
pd.DataFrame(schedule_house_B_occ_1, columns = ["Schedule"], dtype='int64').to_csv("../../data/shatter/Partial_DBSCAN_House-B_Occ-1_SHATTER_Schedule.csv", index = False)

sat
0 20
Ulimate Cost 0 20
Num_timeslot 10 Optimal_cost 20
schedule_0 2
schedule_1 2
schedule_2 2
schedule_3 2
schedule_4 2
schedule_5 2
schedule_6 2
schedule_7 2
schedule_8 2
schedule_9 2
stay_0 1
stay_1 2
stay_2 3
stay_3 4
stay_4 5
stay_5 6
stay_6 7
stay_7 8
stay_8 9
stay_9 10
sat
10 20
Ulimate Cost 10 40
Num_timeslot 10 Optimal_cost 20
schedule_10 2
schedule_11 2
schedule_12 2
schedule_13 2
schedule_14 2
schedule_15 2
schedule_16 2
schedule_17 2
schedule_18 2
schedule_19 2
stay_10 11
stay_11 12
stay_12 13
stay_13 14
stay_14 15
stay_15 16
stay_16 17
stay_17 18
stay_18 19
stay_19 20
sat
20 21
Ulimate Cost 20 61
Num_timeslot 10 Optimal_cost 21
schedule_20 2
schedule_21 2
schedule_22 2
schedule_23 2
schedule_24 2
schedule_25 2
schedule_26 2
schedule_27 2
schedule_28 2
schedule_29 4
stay_20 21
stay_21 22
stay_22 23
stay_23 24
stay_24 25
stay_25 26
stay_26 27
stay_27 28
stay_28 29
stay_29 1
sat
30 20
Ulimate Cost 30 81
Num_timeslot 10 Optimal_cost 20
schedule_30 2
schedule_31 2
schedule_3

sat
260 10
Ulimate Cost 260 356
Num_timeslot 10 Optimal_cost 10
schedule_260 1
schedule_261 1
schedule_262 1
schedule_263 1
schedule_264 1
schedule_265 1
schedule_266 1
schedule_267 1
schedule_268 1
schedule_269 1
stay_260 178
stay_261 179
stay_262 180
stay_263 181
stay_264 182
stay_265 183
stay_266 184
stay_267 185
stay_268 186
stay_269 187
sat
270 10
Ulimate Cost 270 366
Num_timeslot 10 Optimal_cost 10
schedule_270 1
schedule_271 1
schedule_272 1
schedule_273 1
schedule_274 1
schedule_275 1
schedule_276 1
schedule_277 1
schedule_278 1
schedule_279 1
stay_270 188
stay_271 189
stay_272 190
stay_273 191
stay_274 192
stay_275 193
stay_276 194
stay_277 195
stay_278 196
stay_279 197
sat
280 10
Ulimate Cost 280 376
Num_timeslot 10 Optimal_cost 10
schedule_280 1
schedule_281 1
schedule_282 1
schedule_283 1
schedule_284 1
schedule_285 1
schedule_286 1
schedule_287 1
schedule_288 1
schedule_289 1
stay_280 198
stay_281 199
stay_282 200
stay_283 201
stay_284 202
stay_285 203
stay_286 204
stay_28

sat
500 10
Ulimate Cost 500 596
Num_timeslot 10 Optimal_cost 10
schedule_500 1
schedule_501 1
schedule_502 1
schedule_503 1
schedule_504 1
schedule_505 1
schedule_506 1
schedule_507 1
schedule_508 1
schedule_509 1
stay_500 418
stay_501 419
stay_502 420
stay_503 421
stay_504 422
stay_505 423
stay_506 424
stay_507 425
stay_508 426
stay_509 427
sat
510 10
Ulimate Cost 510 606
Num_timeslot 10 Optimal_cost 10
schedule_510 1
schedule_511 1
schedule_512 1
schedule_513 1
schedule_514 1
schedule_515 1
schedule_516 1
schedule_517 1
schedule_518 1
schedule_519 1
stay_510 428
stay_511 429
stay_512 430
stay_513 431
stay_514 432
stay_515 433
stay_516 434
stay_517 435
stay_518 436
stay_519 437
sat
520 10
Ulimate Cost 520 616
Num_timeslot 10 Optimal_cost 10
schedule_520 1
schedule_521 1
schedule_522 1
schedule_523 1
schedule_524 1
schedule_525 1
schedule_526 1
schedule_527 1
schedule_528 1
schedule_529 1
stay_520 438
stay_521 439
stay_522 440
stay_523 441
stay_524 442
stay_525 443
stay_526 444
stay_52

sat
750 0
Ulimate Cost 750 1195
Num_timeslot 10 Optimal_cost 0
schedule_750 0
schedule_751 0
schedule_752 0
schedule_753 0
schedule_754 0
schedule_755 0
schedule_756 0
schedule_757 0
schedule_758 0
schedule_759 0
stay_750 2
stay_751 3
stay_752 4
stay_753 5
stay_754 6
stay_755 7
stay_756 8
stay_757 9
stay_758 10
stay_759 11
sat
760 0
Ulimate Cost 760 1195
Num_timeslot 10 Optimal_cost 0
schedule_760 0
schedule_761 0
schedule_762 0
schedule_763 0
schedule_764 0
schedule_765 0
schedule_766 0
schedule_767 0
schedule_768 0
schedule_769 0
stay_760 12
stay_761 13
stay_762 14
stay_763 15
stay_764 16
stay_765 17
stay_766 18
stay_767 19
stay_768 20
stay_769 21
sat
770 0
Ulimate Cost 770 1195
Num_timeslot 10 Optimal_cost 0
schedule_770 0
schedule_771 0
schedule_772 0
schedule_773 0
schedule_774 0
schedule_775 0
schedule_776 0
schedule_777 0
schedule_778 0
schedule_779 0
stay_770 22
stay_771 23
stay_772 24
stay_773 25
stay_774 26
stay_775 27
stay_776 28
stay_777 29
stay_778 30
stay_779 31
sat
780 0

sat
1000 0
Ulimate Cost 1000 1195
Num_timeslot 10 Optimal_cost 0
schedule_1000 0
schedule_1001 0
schedule_1002 0
schedule_1003 0
schedule_1004 0
schedule_1005 0
schedule_1006 0
schedule_1007 0
schedule_1008 0
schedule_1009 0
stay_1000 252
stay_1001 253
stay_1002 254
stay_1003 255
stay_1004 256
stay_1005 257
stay_1006 258
stay_1007 259
stay_1008 260
stay_1009 261
sat
1010 0
Ulimate Cost 1010 1195
Num_timeslot 10 Optimal_cost 0
schedule_1010 0
schedule_1011 0
schedule_1012 0
schedule_1013 0
schedule_1014 0
schedule_1015 0
schedule_1016 0
schedule_1017 0
schedule_1018 0
schedule_1019 0
stay_1010 262
stay_1011 263
stay_1012 264
stay_1013 265
stay_1014 266
stay_1015 267
stay_1016 268
stay_1017 269
stay_1018 270
stay_1019 271
sat
1020 0
Ulimate Cost 1020 1195
Num_timeslot 10 Optimal_cost 0
schedule_1020 0
schedule_1021 0
schedule_1022 0
schedule_1023 0
schedule_1024 0
schedule_1025 0
schedule_1026 0
schedule_1027 0
schedule_1028 0
schedule_1029 0
stay_1020 272
stay_1021 273
stay_1022 274
sta

sat
1230 0
Ulimate Cost 1230 1195
Num_timeslot 10 Optimal_cost 0
schedule_1230 0
schedule_1231 0
schedule_1232 0
schedule_1233 0
schedule_1234 0
schedule_1235 0
schedule_1236 0
schedule_1237 0
schedule_1238 0
schedule_1239 0
stay_1230 482
stay_1231 483
stay_1232 484
stay_1233 485
stay_1234 486
stay_1235 487
stay_1236 488
stay_1237 489
stay_1238 490
stay_1239 491
sat
1240 0
Ulimate Cost 1240 1195
Num_timeslot 10 Optimal_cost 0
schedule_1240 0
schedule_1241 0
schedule_1242 0
schedule_1243 0
schedule_1244 0
schedule_1245 0
schedule_1246 0
schedule_1247 0
schedule_1248 0
schedule_1249 0
stay_1240 492
stay_1241 493
stay_1242 494
stay_1243 495
stay_1244 496
stay_1245 497
stay_1246 498
stay_1247 499
stay_1248 500
stay_1249 501
sat
1250 0
Ulimate Cost 1250 1195
Num_timeslot 10 Optimal_cost 0
schedule_1250 0
schedule_1251 0
schedule_1252 0
schedule_1253 0
schedule_1254 0
schedule_1255 0
schedule_1256 0
schedule_1257 0
schedule_1258 0
schedule_1259 0
stay_1250 502
stay_1251 503
stay_1252 504
sta

# House-B Occupant-1 Greedy Schedule

In [19]:
greedy_schedule_house_B_occ_1 = greedy_schedule(clusters_house_B_occ_1, 1440)
pd.DataFrame(greedy_schedule_house_B_occ_1, columns = ["Schedule"], dtype='int64').to_csv("../../data/shatter/Partial_DBSCAN_House-B_Occ-1_Greedy_Schedule.csv", index = False)

Here 0 61
selected zone 2
Here 61 63
selected zone 4
Here 63 65
selected zone 4
Here 65 67
selected zone 4
Here 67 69
selected zone 4
Here 69 71
selected zone 4
Here 71 73
selected zone 4
Here 73 75
selected zone 4
Here 75 77
selected zone 4
Here 77 79
selected zone 4
Here 79 81
selected zone 4
Here 81 83
selected zone 4
Here 84 575
selected zone 1
Here 575 585
selected zone 4
Here 585 595
selected zone 4
Here 595 605
selected zone 4
Here 605 615
selected zone 4
Here 615 625
selected zone 4
Here 625 634
selected zone 4
Here 634 643
selected zone 4
Here 643 652
selected zone 4
Here 652 682
selected zone 3
Here 682 691
selected zone 4
Here 691 700
selected zone 4
Here 700 709
selected zone 4
Here 709 718
selected zone 4
Here 718 727
selected zone 4
Here 727 734
selected zone 4
Here 734 739
selected zone 4
Here 739 742
selected zone 4
Here 742 745
selected zone 4
Here 745 747
selected zone 4
Here 747 748
selected zone 4
Here 748 749
selected zone 4
Critical 749
Critical 750
Critical 751
C

# House-B Occupant-2 Attack Schedule

In [20]:
schedule_house_B_occ_2 = attack_scheduling(clusters_house_B_occ_2, num_timeslots, start_time, final_time)
pd.DataFrame(schedule_house_B_occ_2, columns = ["Schedule"], dtype='int64').to_csv("../../data/shatter/Partial_DBSCAN_House-B_Occ-2_SHATTER_Schedule.csv", index = False)

sat
0 0
Ulimate Cost 0 0
Num_timeslot 10 Optimal_cost 0
schedule_0 0
schedule_1 0
schedule_2 0
schedule_3 0
schedule_4 0
schedule_5 0
schedule_6 0
schedule_7 0
schedule_8 0
schedule_9 0
stay_0 1
stay_1 2
stay_2 3
stay_3 4
stay_4 5
stay_5 6
stay_6 7
stay_7 8
stay_8 9
stay_9 10
sat
10 0
Ulimate Cost 10 0
Num_timeslot 10 Optimal_cost 0
schedule_10 0
schedule_11 0
schedule_12 0
schedule_13 0
schedule_14 0
schedule_15 0
schedule_16 0
schedule_17 0
schedule_18 0
schedule_19 0
stay_10 11
stay_11 12
stay_12 13
stay_13 14
stay_14 15
stay_15 16
stay_16 17
stay_17 18
stay_18 19
stay_19 20
sat
20 0
Ulimate Cost 20 0
Num_timeslot 10 Optimal_cost 0
schedule_20 0
schedule_21 0
schedule_22 0
schedule_23 0
schedule_24 0
schedule_25 0
schedule_26 0
schedule_27 0
schedule_28 0
schedule_29 0
stay_20 21
stay_21 22
stay_22 23
stay_23 24
stay_24 25
stay_25 26
stay_26 27
stay_27 28
stay_28 29
stay_29 30
sat
30 0
Ulimate Cost 30 0
Num_timeslot 10 Optimal_cost 0
schedule_30 0
schedule_31 0
schedule_32 0
schedul

sat
260 0
Ulimate Cost 260 0
Num_timeslot 10 Optimal_cost 0
schedule_260 0
schedule_261 0
schedule_262 0
schedule_263 0
schedule_264 0
schedule_265 0
schedule_266 0
schedule_267 0
schedule_268 0
schedule_269 0
stay_260 261
stay_261 262
stay_262 263
stay_263 264
stay_264 265
stay_265 266
stay_266 267
stay_267 268
stay_268 269
stay_269 270
sat
270 0
Ulimate Cost 270 0
Num_timeslot 10 Optimal_cost 0
schedule_270 0
schedule_271 0
schedule_272 0
schedule_273 0
schedule_274 0
schedule_275 0
schedule_276 0
schedule_277 0
schedule_278 0
schedule_279 0
stay_270 271
stay_271 272
stay_272 273
stay_273 274
stay_274 275
stay_275 276
stay_276 277
stay_277 278
stay_278 279
stay_279 280
sat
280 0
Ulimate Cost 280 0
Num_timeslot 10 Optimal_cost 0
schedule_280 0
schedule_281 0
schedule_282 0
schedule_283 0
schedule_284 0
schedule_285 0
schedule_286 0
schedule_287 0
schedule_288 0
schedule_289 0
stay_280 281
stay_281 282
stay_282 283
stay_283 284
stay_284 285
stay_285 286
stay_286 287
stay_287 288
stay_2

sat
510 20
Ulimate Cost 510 136
Num_timeslot 10 Optimal_cost 20
schedule_510 0
schedule_511 4
schedule_512 1
schedule_513 4
schedule_514 4
schedule_515 0
schedule_516 4
schedule_517 4
schedule_518 4
schedule_519 1
stay_510 30
stay_511 1
stay_512 1
stay_513 1
stay_514 2
stay_515 1
stay_516 1
stay_517 2
stay_518 3
stay_519 1
sat
520 20
Ulimate Cost 520 156
Num_timeslot 10 Optimal_cost 20
schedule_520 1
schedule_521 4
schedule_522 4
schedule_523 1
schedule_524 1
schedule_525 4
schedule_526 4
schedule_527 1
schedule_528 1
schedule_529 4
stay_520 2
stay_521 1
stay_522 2
stay_523 1
stay_524 2
stay_525 1
stay_526 2
stay_527 1
stay_528 2
stay_529 1
sat
530 15
Ulimate Cost 530 171
Num_timeslot 10 Optimal_cost 15
schedule_530 0
schedule_531 4
schedule_532 1
schedule_533 1
schedule_534 4
schedule_535 1
schedule_536 1
schedule_537 4
schedule_538 1
schedule_539 1
stay_530 1
stay_531 1
stay_532 1
stay_533 2
stay_534 1
stay_535 1
stay_536 2
stay_537 1
stay_538 1
stay_539 2
sat
540 10
Ulimate Cost 540

sat
760 0
Ulimate Cost 760 610
Num_timeslot 10 Optimal_cost 0
schedule_760 0
schedule_761 0
schedule_762 0
schedule_763 0
schedule_764 0
schedule_765 0
schedule_766 0
schedule_767 0
schedule_768 0
schedule_769 0
stay_760 49
stay_761 50
stay_762 51
stay_763 52
stay_764 53
stay_765 54
stay_766 55
stay_767 56
stay_768 57
stay_769 58
sat
770 0
Ulimate Cost 770 610
Num_timeslot 10 Optimal_cost 0
schedule_770 0
schedule_771 0
schedule_772 0
schedule_773 0
schedule_774 0
schedule_775 0
schedule_776 0
schedule_777 0
schedule_778 0
schedule_779 0
stay_770 59
stay_771 60
stay_772 61
stay_773 62
stay_774 63
stay_775 64
stay_776 65
stay_777 66
stay_778 67
stay_779 68
sat
780 0
Ulimate Cost 780 610
Num_timeslot 10 Optimal_cost 0
schedule_780 0
schedule_781 0
schedule_782 0
schedule_783 0
schedule_784 0
schedule_785 0
schedule_786 0
schedule_787 0
schedule_788 0
schedule_789 0
stay_780 69
stay_781 70
stay_782 71
stay_783 72
stay_784 73
stay_785 74
stay_786 75
stay_787 76
stay_788 77
stay_789 78
sat


sat
1010 0
Ulimate Cost 1010 610
Num_timeslot 10 Optimal_cost 0
schedule_1010 0
schedule_1011 0
schedule_1012 0
schedule_1013 0
schedule_1014 0
schedule_1015 0
schedule_1016 0
schedule_1017 0
schedule_1018 0
schedule_1019 0
stay_1010 299
stay_1011 300
stay_1012 301
stay_1013 302
stay_1014 303
stay_1015 304
stay_1016 305
stay_1017 306
stay_1018 307
stay_1019 308
sat
1020 0
Ulimate Cost 1020 610
Num_timeslot 10 Optimal_cost 0
schedule_1020 0
schedule_1021 0
schedule_1022 0
schedule_1023 0
schedule_1024 0
schedule_1025 0
schedule_1026 0
schedule_1027 0
schedule_1028 0
schedule_1029 0
stay_1020 309
stay_1021 310
stay_1022 311
stay_1023 312
stay_1024 313
stay_1025 314
stay_1026 315
stay_1027 316
stay_1028 317
stay_1029 318
sat
1030 0
Ulimate Cost 1030 610
Num_timeslot 10 Optimal_cost 0
schedule_1030 0
schedule_1031 0
schedule_1032 0
schedule_1033 0
schedule_1034 0
schedule_1035 0
schedule_1036 0
schedule_1037 0
schedule_1038 0
schedule_1039 0
stay_1030 319
stay_1031 320
stay_1032 321
stay_1

sat
1240 10
Ulimate Cost 1240 640
Num_timeslot 10 Optimal_cost 10
schedule_1240 1
schedule_1241 1
schedule_1242 1
schedule_1243 1
schedule_1244 1
schedule_1245 1
schedule_1246 1
schedule_1247 1
schedule_1248 1
schedule_1249 1
stay_1240 7
stay_1241 8
stay_1242 9
stay_1243 10
stay_1244 11
stay_1245 12
stay_1246 13
stay_1247 14
stay_1248 15
stay_1249 16
sat
1250 8
Ulimate Cost 1250 648
Num_timeslot 10 Optimal_cost 8
schedule_1250 1
schedule_1251 1
schedule_1252 1
schedule_1253 1
schedule_1254 1
schedule_1255 1
schedule_1256 1
schedule_1257 1
schedule_1258 0
schedule_1259 0
stay_1250 17
stay_1251 18
stay_1252 19
stay_1253 20
stay_1254 21
stay_1255 22
stay_1256 23
stay_1257 24
stay_1258 1
stay_1259 2
sat
1260 0
Ulimate Cost 1260 648
Num_timeslot 10 Optimal_cost 0
schedule_1260 0
schedule_1261 0
schedule_1262 0
schedule_1263 0
schedule_1264 0
schedule_1265 0
schedule_1266 0
schedule_1267 0
schedule_1268 0
schedule_1269 0
stay_1260 3
stay_1261 4
stay_1262 5
stay_1263 6
stay_1264 7
stay_1265 8

# House-B Occupant-2 Greedy Schedule

In [21]:
greedy_schedule_house_B_occ_2 = greedy_schedule(clusters_house_B_occ_2, 1440)
pd.DataFrame(greedy_schedule_house_B_occ_2, columns = ["Schedule"], dtype='int64').to_csv("../../data/shatter/Partial_DBSCAN_House-B_Occ-2_Greedy_Schedule.csv", index = False)

Critical 0
Critical 1
Critical 2
Critical 3
Critical 4
Critical 5
Critical 6
Critical 7
Critical 8
Critical 9
Critical 10
Critical 11
Critical 12
Critical 13
Critical 14
Critical 15
Critical 16
Critical 17
Critical 18
Critical 19
Critical 20
Critical 21
Critical 22
Critical 23
Critical 24
Critical 25
Critical 26
Critical 27
Critical 28
Critical 29
Critical 30
Critical 31
Critical 32
Critical 33
Critical 34
Critical 35
Critical 36
Critical 37
Critical 38
Critical 39
Critical 40
Critical 41
Critical 42
Critical 43
Critical 44
Critical 45
Critical 46
Critical 47
Critical 48
Critical 49
Critical 50
Critical 51
Critical 52
Critical 53
Critical 54
Critical 55
Critical 56
Critical 57
Critical 58
Critical 59
Critical 60
Critical 61
Critical 62
Critical 63
Critical 64
Critical 65
Critical 66
Critical 67
Critical 68
Critical 69
Critical 70
Critical 71
Critical 72
Critical 73
Critical 74
Critical 75
Critical 76
Critical 77
Critical 78
Critical 79
Critical 80
Critical 81
Critical 82
Critical 83
Cr